In [9]:
import pandas as pd
import spacy
import random 
import json
import os
from pathlib import Path
from spacy.training import Example
from spacy.tokens import DocBin
from spacy.language import Language
from spacy.tokens import Span
from spacy.pipeline import EntityRuler
from spacy.matcher import PhraseMatcher
from sklearn.model_selection import train_test_split



os.chdir(r'/home/dlin1765/repos/tekken-move-guesser')
with open('./annotations/v3-annotations.json', 'r', encoding='utf-8') as raw:
    data = json.load(raw)

with open('./annotations/v3-test-annotations.json', 'r', encoding='utf-8') as raw:
    testData = json.load(raw)


In [10]:
import json
from types import NoneType
# create new training data 
limbList = ['punch', 'jab', 'palm', 'fist', 'uppercut', 'hand', 'chop', 'slap', 'armbar', 'swing', 'shove', 'push', 'elbow', 'shoulder', 'tackle', 'body', 'head', 'headbutt', 'finger', 'butt', 'leg', 'foot', 'kick', 'roundhouse', 'round house', 'axe kick', 'axekick', 'heel', 'sweep', 'slide', 'knee']
specialLimbList = ['sword', 'dagger', 'gun', 'pistol', 'handgun', 'nunchuck', 'cane', 'chainsaw', 'chain saw', 'chain-saw', 'magic', 'energy', 'flip', 'backflip', 'cartwheel', 'cart wheel']
howListPre = ['jumping', 'bouncing', 'hopping','running', 'spinning', 'approaching', 'advancing', 'forward moving', 'backwards moving', 'backwards', 'retreating', 'dodging', 'evading','flying', 'rolling', 'sliding', 'overhead', 'downwards', 'rising', 'forward']
howListDescriptors = ['big', 'long', 'long range', 'giant', 'huge', 'fast', 'slow', 'quick', 'rapid','reactable']
howListPost = ['strike', 'move', 'poke', 'check', 'smash', 'jab', 'move', 'move', 'move', 'move']
specialHowList = ['swing', 'stab', 'blast', 'strike', 'move', 'poke', 'check', 'smash', 'jab', 'move', 'move', 'move', 'move', 'slash', 'beam', 'thrust']

tekkenPropertyDescriptors = ['plus on block', 'plus frame', 'plus', 'safe', 'minus on block', 'minus', 'unsafe', 'punishable']
tekkenProperties = ['high', 'low', 'mid', 'guard crush', 'unblockable', 'untechable', 'launcher', 'stance', 'tornado', 'knockdown', 'knock down', 'stagger', 'high crush', 'low crush', 'high crushing', 'low crushing', 'evasive', 'homing'] # chargable, holdable, 
tekkenPropertiesPost = ['guard crushes', 'launches', 'tornados', 'knocks down', 'staggers', 'high crushes', 'low crushes', 'evades']
tekkenPropertiesPost2 = ['guard crushes', 'launches', 'tornados', 'knocks down', 'staggers', 'high crushes', 'low crushes', 'evades', 'hits grounded', 'is chargable', 'is holdable', 'transitions to stance', 'transitions into a stance', 'transitions into stance', 'goes into stance', 'leaves them in stance']
launching = [ 'guard crushing', 'unblockable', 'untechable', 'launcher', 'launching','stance', 'tornado', 'knockdown', 'knock down', 'staggering', 'high crushing', 'low crushing', 'evasive', 'homing']
tekkenMoveTypes = ['heat engager', 'heat smash', 'heat burst', 'parry', 'grab', 'power crush', 'armor', 'armored', 'throw', 'chain throw', 'chain grab', 'homing']


string_detail = ['two hit', 'three hit', 'four hit', 'five hit', 'multihit', 'multi hit', 'two part', 'three part', 'four part', 'five part']
string_detailPost = ['first hit', 'last hit', 'starts with', 'begins with', 'ends with', 'finishes with']
string_detailPost2 = ['first', 'second', 'third', 'last', 'starts', 'begins', 'ends', 'finishes']   
actions = ['punches', 'kicks', 'palms', 'jabs', 'uppercuts', 'chops', 'slaps', 'swings', 'pushes', 'shoves', 'elbows', 'knees', 'headbutts', 'butts', 'tackles', 'sweeps', 'slides']

# LIMB SPECIAL_LIMB LIMB_HOW LIMB_DESCRIPTION MOVE_TYPE MOVE_PROPERTY STRING_DETAIL

validDict = {
    "LIMB": limbList+specialLimbList, 
    "SPECIAL_LIMB": specialLimbList, 
    "LIMB_HOW": howListPre, 
    "LIMB_DESCRIPTION": howListDescriptors, 
    "MOVE_TYPE": tekkenMoveTypes, 
    "MOVE_PROPERTY": tekkenPropertyDescriptors+tekkenProperties+tekkenPropertiesPost+launching, 
    "STRING_DETAIL": ['string', 'strings'] + string_detail + string_detailPost2+ ['one', 'two', 'three', 'four', 'five', 'multi']
} 

#{"classes": ["LIMB", "SPECIAL_LIMB", "LIMB_HOW", "LIMB_DESCRIPTION", "MOVE_TYPE", "MOVE_PROPERTY", "STRING_DETAIL"], "annotations": null}

def generateStrings2(listOfWords, label1, label2):
    # for each list in the list of words 
    count = 0
    random.seed(10)
    train_data = open("./raw-data/traindata3.txt","a")
    test_data = open('testdata2.txt', "a")
    allAnnotatedLists = []
    for x in range(0, len(listOfWords)):
        for y in range(x+1, len(listOfWords)):
            for word1 in listOfWords[x]:
                for word2 in listOfWords[y]:
                    if(count % 5 != 0):
                        newString = word1 + " " + word2
                        #train_data.write(newString + "\n")
                        labeledList1 = annotateData(0, word1, label1)
                        labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                        annotatedList = [labeledList1, labeledList2]
                        allAnnotatedLists.append([newString, {"entities": annotatedList}])
                    else:
                        newString = word1 + " " + word2 + ' ' + howListPost[random.randrange(0,10)]
                        #train_data.write(newString + "\n")
                        labeledList1 = annotateData(0, word1, label1)
                        labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                        annotatedList = [labeledList1, labeledList2]
                        allAnnotatedLists.append([newString, {"entities": annotatedList}])
                    count += 1
    #addJSONData(allAnnotatedLists)
    addDataToJSONandTxt(allAnnotatedLists)

def generateStrings2a(listOfWords, label1, label2):
    # for each list in the list of words 
    count = 0
    random.seed(10)
    train_data = open("traindata2.txt","a")
    test_data = open('testdata2.txt', "a")
    allAnnotatedLists = []
    for x in range(0, len(listOfWords)):
        for y in range(x+1, len(listOfWords)):
            for word1 in listOfWords[x]:
                for word2 in listOfWords[y]:
                    if(count % 5 != 0):
                        newString = word1 + " that " + word2
                        #train_data.write(newString + "\n")
                        labeledList1 = annotateData(0, word1, label1)
                        labeledList2 = annotateData(len(word1 +" that "), word2 , label2)
                        annotatedList = [labeledList1, labeledList2]
                        allAnnotatedLists.append([newString, {"entities": annotatedList}])
                    else:
                        newString = word1 + " " + howListPost[random.randrange(0,10)] + " that " + word2
                        #train_data.write(newString + "\n")
                        labeledList1 = annotateData(0, word1, label1)
                        labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                        annotatedList = [labeledList1, labeledList2]
                        allAnnotatedLists.append([newString, {"entities": annotatedList}])
                    count += 1
    #addJSONData(allAnnotatedLists)
    addDataToJSONandTxt(allAnnotatedLists)

def generateStrings2String(listOfWords, label1, label2):
    count = 0
    random.seed(10)
    allAnnotatedLists = []
    for x in range(0, len(listOfWords)):
        for y in range(x+1, len(listOfWords)):
            for word1 in listOfWords[x]:
                for word2 in listOfWords[y]:
                    if(count % 5 != 0):
                        newString = word1 + " " + word2 + " string"
                        #train_data.write(newString + "\n")
                        labeledList1 = annotateData(0, word1, label1)
                        labeledList2 = annotateData(len(word1 + " "), word2 , label2)
                        labeledList3 = annotateData(len(word1 + " " + word2 + " "), 'string', 'STRING_DETAIL')
                        annotatedList = [labeledList1, labeledList2, labeledList3]
                        allAnnotatedLists.append([newString, {"entities": annotatedList}])
                    else:
                        newString = word1 + " "  + word2 + " string"
                        #train_data.write(newString + "\n")
                        labeledList1 = annotateData(0, word1, label1)
                        labeledList2 = annotateData(len(word1 + " "), word2 , label2)
                        labeledList3 = annotateData(len(word1 + " " + word2 + " "), 'string', 'STRING_DETAIL')
                        annotatedList = [labeledList1, labeledList2, labeledList3]
                        allAnnotatedLists.append([newString, {"entities": annotatedList}])
                    count += 1
    #addJSONData(allAnnotatedLists)
    addDataToJSONandTxt(allAnnotatedLists)

def generateStrings3(listA, listB, listC, label1, label2, label3):
    count = 0
    random.seed(10)
    allAnnotatedLists = []
    for word1 in listA:
        for word2 in listB:
            for word3 in listC:
                if count % 5 != 0:
                    newString = word1 + " " + word2 + " " + word3
                    labeledList1 = annotateData(0, word1, label1)
                    labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                    labeledList3 = annotateData(len(word1 + " " + word2 + " "), word3, label3)
                    annotatedList = [labeledList1, labeledList2, labeledList3]
                    allAnnotatedLists.append([newString, {"entities": annotatedList}])
                else:
                    newString = word1 + " " + word2 + " " + word3 + " " + howListPost[random.randrange(0,10)]
                    labeledList1 = annotateData(0, word1, label1)
                    labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                    labeledList3 = annotateData(len(word1 + " " + word2 + " "), word3, label3)
                    annotatedList = [labeledList1, labeledList2, labeledList3]
                    allAnnotatedLists.append([newString, {"entities": annotatedList}])
                count += 1
    addDataToJSONandTxt(allAnnotatedLists)

def generateStrings3a(listA, listB, listC, label1, label2, label3):
    count = 0
    random.seed(10)
    allAnnotatedLists = []
    for word1 in listA:
        for word2 in listB:
            for word3 in listC:
                if count % 5 != 0:
                    newString = word1 + " " + word2 + " that " + word3
                    labeledList1 = annotateData(0, word1, label1)
                    labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                    labeledList3 = annotateData(len(word1 + " " + word2 + " that "), word3, label3)
                    annotatedList = [labeledList1, labeledList2, labeledList3]
                    allAnnotatedLists.append([newString, {"entities": annotatedList}]) 
                else:
                    newString = word1 + " " + word2 + " move" +" that " + word3 
                    labeledList1 = annotateData(0, word1, label1)
                    labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                    labeledList3 = annotateData(len(word1 + " " + word2 + " move that "), word3, label3)
                    annotatedList = [labeledList1, labeledList2, labeledList3]
                    allAnnotatedLists.append([newString, {"entities": annotatedList}]) 
                count += 1
    addDataToJSONandTxt(allAnnotatedLists)

def generateStrings3String(listA, listB, listC, label1, label2, label3):
    random.seed(10)
    allAnnotatedLists = []
    for word1 in listA:
        for word2 in listB:
            for word3 in listC:
                newString = word1 + " " + word2 + " " + word3 + " string"
                labeledList1 = annotateData(0, word1, label1)
                labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                labeledList3 = annotateData(len(word1 + " " + word2 + " "), word3, label3)
                labeledList4 = annotateData(len(word1 + " " + word2 + " " + word3 + " "), 'string', 'STRING_DETAIL')
                annotatedList = [labeledList1, labeledList2, labeledList3, labeledList4]
                allAnnotatedLists.append([newString, {"entities": annotatedList}])
    addDataToJSONandTxt(allAnnotatedLists)

# move where they do a approaching punch into a reactable punch giant elbow
def generateStringsStringPre(listA, listB, listC, label1, label2, label3):
    # move/string where he/she/they do a* punch (into a)* punch (into a)* punch
    random.seed(10)
    allAnnotatedLists = []
    stringMove = ['string', 'move']
    pronoun = ['he', 'she', 'they']
    for word1 in listA:
        for word2 in listB:
            for word3 in listC:
                pronounNum = random.randint(0,2)
                if pronounNum == 2:
                    transition = " do a"
                else:
                    transition = " does a"
                newString = stringMove[random.randint(0,1)] + " where " + pronoun[pronounNum] + transition
                
                big1 = randomWordChance(0,1,1,howListDescriptors, "")
                big2 = randomWordChance(0,1,1,howListDescriptors, "")
                big3 = randomWordChance(0,1,1,howListDescriptors, "")

                jumping1 = randomWordChance(0,1,1,howListPre, "")
                jumping2 = randomWordChance(0,1,1,howListPre, "")
                jumping3 = randomWordChance(0,1,1,howListPre, "")

                intoA1 = randomWordChance(0,1,1, ['into a'], "")
                intoA2 = randomWordChance(0,1,1, ['into a', 'into another', 'then a'], "")
                newString1 = newString + big1 + jumping1 + " " + word1 + intoA1 + big2 + jumping2 + " " + word2 + intoA2 + big3 + jumping3 + " " + word3
                #print(newString)
                splitUp = newString.split(" ")
                annotatedList = []
                if(splitUp[0] == "string"):
                    annotatedList.append(annotateData(0, 'string', 'STRING_DETAIL'))
                if(big1 != ""):
                    annotatedList.append(annotateData(len(newString+ " "), big1[1:] , 'LIMB_DESCRIPTION'))
                if(jumping1 != ""):
                    annotatedList.append(annotateData(len(newString + big1 + " "), jumping1[1:], 'LIMB_HOW'))

                annotatedList.append(annotateData(len(newString + big1 + jumping1 + " "), word1 , 'LIMB'))
                
                if(big2 != ""):
                    annotatedList.append(annotateData(len(newString + big1 + jumping1 + " " + word1 + intoA1 + " "), big2[1:], 'LIMB_DESCRIPTION'))
                if(jumping2 != ""):
                    annotatedList.append(annotateData(len(newString + big1 + jumping1 + " " + word1 + intoA1 + big2 + " "), jumping2[1:], 'LIMB_HOW'))

                annotatedList.append(annotateData(len(newString + big1 + jumping1 + " " + word1 + intoA1 + big2 + jumping2 + " "), word2 , 'LIMB'))

                if(big3 != ""):
                    annotatedList.append(annotateData(len(newString + big1 + jumping1 + " " + word1 + intoA1 + big2 + jumping2 + " " + word2 + intoA2 + " "), big3[1:], 'LIMB_DESCRIPTION'))
                if(jumping3 != ""):
                    annotatedList.append(annotateData(len(newString + big1 + jumping1 + " " + word1 + intoA1 + big2 + jumping2 + " " + word2 + intoA2 + big3 + " "), jumping3[1:], 'LIMB_HOW'))
                
                annotatedList.append(annotateData(len(newString + big1 + jumping1 + " " + word1 + intoA1 + big2 + jumping2 + " " + word2 + intoA2 + big3 + jumping3 + " "), word3 , 'LIMB'))
             
                #print([newString1, {"entities": annotatedList}])
                validateAnnotations([newString1, {"entities": annotatedList}], validDict)
                allAnnotatedLists.append([newString1, {"entities": annotatedList}])
    addDataToJSONandTxt(allAnnotatedLists)
    return

# two hit (mid/high) string 
# two hit punch kick mid low string 
# kick punch string that starts with low 

# strings/string move/string with.../ that have (number) lows
# (tekken property) move/moves/strings/string move/string with.../ that start with a punch and ends with a (low/mid/high) punch
# string with three hits that launches on hit 
# three hit string where the last hit launches and has a low extension 
# launching three hit string with a low extension 
# move 
# multi hit safe/spinning/flying string with punches and kicks 

# move where he/she/they   


def generateStringsSpecific(listA, listB, listC, label1, label2, label3):
    # safe/launching two hit punch kick mid low string 
    # two hit punch kick mid low string that launches 
    random.seed(10)
    string_detail = ['two hit', 'three hit', 'four hit', 'five hit', 'multihit', 'multi hit', 'two part', 'three part', 'four part', 'five part', '']
    string_detailPost = ['first hit', 'second hit', 'third hit', 'last hit', 'starts with', 'begins with', 'ends with', 'finishes with']   
    plus = ['plus on block', 'plus frame', 'plus', 'safe', 'minus on block', 'minus', 'unsafe', 'punishable']
    punctuation = [', ', " "]
    
    launches = ['guard crushes', 'launches', 'tornados', 'knocks down', 'staggers', 'high crushes', 'low crushes', 'evades', 'homes', 'is chargable', 'is holdable', 'hits grounded']
    hitTypes = ['mid', 'high', 'low', 'unblockable', 'mid', 'high', 'low','mid', 'high', 'low', 'mid', 'high', 'low']
    allAnnotatedLists = []

    superLimbList = limbList + specialLimbList + ['punch', 'punch', 'punch', 'punch', 'punch', 'punch', 'punch', 'punch', 'punch', 'punch', 'punch', 'punch', 'punch', 'punch', 'kick','kick','kick','kick','kick','kick','kick','kick','kick','kick','kick','kick','kick','kick','kick']

    # safe / launching (hit count) (hit types) string that starts with a low that high crushes/where the first hit is a low that knocks down/ 

    for word1 in listA:
        for word2 in listB:
            for word3 in listC:
                # both/just safe/just launching/nothing
                annotatedList = []
                newString = ""
                prefixNum = random.randint(0,3)
                if prefixNum == 0:
                    order = random.randint(0,1)
                    if order == 0:
                        prefix = plus[random.randint(0, len(plus) - 1)] 
                        annotatedList.append(annotateData(0, prefix, 'MOVE_PROPERTY'))
                        newString = newString + prefix + punctuation[random.randint(0, len(punctuation) -1)]
                        prop = launching[random.randint(0, len(launching) - 1)]
                        annotatedList.append(annotateData(len(newString), prop, 'MOVE_PROPERTY'))
                        newString = newString + prop + " "
                    else:
                        prop = launching[random.randint(0, len(launching) - 1)]
                        annotatedList.append(annotateData(0, prop, 'MOVE_PROPERTY'))
                        newString = newString + prop + punctuation[random.randint(0, len(punctuation) -1)]
                        frames = plus[random.randint(0, len(plus) - 1)] 
                        annotatedList.append(annotateData(len(newString), frames, 'MOVE_PROPERTY'))
                        newString = newString + frames + " "
                elif prefixNum == 1:
                    prefix = plus[random.randint(0, len(plus) - 1)] 
                    annotatedList.append(annotateData(0, prefix, 'MOVE_PROPERTY'))
                    newString = newString + prefix + " "
                elif prefixNum == 2:
                    prop = launching[random.randint(0, len(launching) - 1)]
                    annotatedList.append(annotateData(0, prop, 'MOVE_PROPERTY'))
                    newString = newString + prop + " "
                if(len(word1)!= 0):
                        word = word1.split(" ")
                        annotatedList.append(annotateData(len(newString), word[0], 'STRING_DETAIL'))
                        newString = newString + word1 + " "
                # 1/4 chance to add limbs to the string 
                if random.randint(0,5) == 5:
                    numHits = random.randint(1,5)
                    i = 0
                    punctuationNum = random.randint(0,2)
                    hits = ""
                    while i < numHits:
                        newHit = superLimbList[random.randint(0, len(superLimbList) -1)]
                        if(punctuationNum == 0):
                            if i + 1 == numHits:
                                annotatedList.append(annotateData(len(newString), newHit, 'LIMB'))
                                newString = newString + newHit + " "
                            else:
                                annotatedList.append(annotateData(len(newString), newHit, 'LIMB'))
                                newString = newString + newHit + ", "
                        else:
                            annotatedList.append(annotateData(len(newString), newHit, 'LIMB'))
                            newString = newString + newHit + " "
                        i += 1
                # add string/move/strings/moves to the string
                strList = ['string', 'strings']
                moveList = ['move', 'moves']
                strMoveNum = random.randint(0,1)
                if strMoveNum == 0:
                    word = strList[random.randint(0, len(strList) - 1)]
                    if len(word) != 0:
                        annotatedList.append(annotateData(len(newString), word, 'STRING_DETAIL'))
                    newString = newString + word + " "
                else:
                    word = moveList[random.randint(0, len(moveList) - 1)]
                    newString = newString + word + " "
                # string that starts with a (safe) (jumping) (mid) (kick)
                if word2.split(" ")[1] == 'with':
                    newString = newString + "that "
                    word = word2.split(" ")
                    annotatedList.append(annotateData(len(newString), word[0], 'STRING_DETAIL'))
                    newString = newString+ word2 + " a "
                else:
                    newString = newString + "where the "
                    word = word2.split(" ")
                    annotatedList.append(annotateData(len(newString), word[0], 'STRING_DETAIL'))
                    newString = newString+ word2 + " is a "
                startNum = random.randint(0, 8)
                match (startNum):
                    case 0:
                        kick = superLimbList[random.randint(0, len(superLimbList) -1)]
                        annotatedList.append(annotateData(len(newString), kick, 'LIMB'))
                        newString = newString + kick + ""
                    case 1: 
                        low = hitTypes[random.randint(0, len(hitTypes) -1)]
                        annotatedList.append(annotateData(len(newString), low, 'MOVE_PROPERTY'))
                        newString = newString + low + ""
                    case 2:
                        low = hitTypes[random.randint(0, len(hitTypes) -1)]
                        annotatedList.append(annotateData(len(newString), low, 'MOVE_PROPERTY'))
                        newString = newString + low + " "
                        kick = superLimbList[random.randint(0, len(superLimbList) -1)]
                        annotatedList.append(annotateData(len(newString), kick, 'LIMB'))
                        newString = newString + kick + ""
                    case 3: 
                        jumping = howListPre[random.randint(0, len(howListPre) - 1)]
                        annotatedList.append(annotateData(len(newString), jumping, 'LIMB_HOW'))
                        newString = newString + jumping + " "
                        low = hitTypes[random.randint(0, len(hitTypes) -1)]
                        annotatedList.append(annotateData(len(newString), low, 'MOVE_PROPERTY'))
                        newString = newString + low + " "
                        kick = superLimbList[random.randint(0, len(superLimbList) -1)]
                        annotatedList.append(annotateData(len(newString), kick, 'LIMB'))
                        newString = newString + kick + ""
                    case 4: 
                        safe = plus[random.randint(0, len(plus) - 1)]
                        annotatedList.append(annotateData(len(newString), safe, 'MOVE_PROPERTY'))
                        newString = newString + safe + " "
                        jumping = howListPre[random.randint(0, len(howListPre) - 1)]
                        annotatedList.append(annotateData(len(newString), jumping, 'LIMB_HOW'))
                        newString = newString + jumping + " "
                        low = hitTypes[random.randint(0, len(hitTypes) -1)]
                        annotatedList.append(annotateData(len(newString), low, 'MOVE_PROPERTY'))
                        newString = newString + low + " "
                        kick = superLimbList[random.randint(0, len(superLimbList) -1)]
                        annotatedList.append(annotateData(len(newString), kick, 'LIMB'))
                        newString = newString + kick + ""
                    case 5:
                        safe = plus[random.randint(0, len(plus) - 1)]
                        annotatedList.append(annotateData(len(newString), safe, 'MOVE_PROPERTY'))
                        newString = newString + safe + " "
                        low = hitTypes[random.randint(0, len(hitTypes) -1)]
                        annotatedList.append(annotateData(len(newString), low, 'MOVE_PROPERTY'))
                        newString = newString + low + ""
                    case 6: 
                        safe = plus[random.randint(0, len(plus) - 1)]
                        annotatedList.append(annotateData(len(newString), safe, 'MOVE_PROPERTY'))
                        newString = newString + safe + " "
                        kick = superLimbList[random.randint(0, len(superLimbList) -1)]
                        annotatedList.append(annotateData(len(newString), kick, 'LIMB'))
                        newString = newString + kick + ""
                    case 7: 
                        jumping = howListPre[random.randint(0, len(howListPre) - 1)]
                        annotatedList.append(annotateData(len(newString), jumping, 'LIMB_HOW'))
                        newString = newString + jumping + " "
                        kick = superLimbList[random.randint(0, len(superLimbList) -1)]
                        annotatedList.append(annotateData(len(newString), kick, 'LIMB'))
                        newString = newString + kick + ""
                    case 8: 
                        jumping = howListPre[random.randint(0, len(howListPre) - 1)]
                        annotatedList.append(annotateData(len(newString), jumping, 'LIMB_HOW'))
                        newString = newString + jumping + " "
                        low = hitTypes[random.randint(0, len(hitTypes) -1)]
                        annotatedList.append(annotateData(len(newString), low, 'MOVE_PROPERTY'))
                        newString = newString + low + ""
                addAdditional = random.randint(0, 1)
                if addAdditional == 0:
                    newString = newString + " that "
                    #annotatedList.append(annotateData(len(newString), word3, 'MOVE_PROPERTY'))
                    newString = newString + word3
                
                print(newString)
                validateAnnotations([newString, {"entities": annotatedList}], validDict)
                allAnnotatedLists.append([newString, {"entities": annotatedList}])
                
    addDataToJSONandTxt(allAnnotatedLists)

                # numHits = random.randint(1,5)
                # i = 0
                # punctuationNum = random.randint(0,2)
                
                # hits = ""
                # while i < numHits:
                #     newHit = hitTypes[random.randint(0, len(hitTypes) -1)]
                #     if(punctuationNum == 0):
                #         if i + 1 == numHits:
                #             annotatedList.append(annotateData(len(newString), newHit, 'MOVE_PROPERTY'))
                #             newString = newString + newHit + " "
                #         else:
                #             annotatedList.append(annotateData(len(newString), newHit, 'MOVE_PROPERTY'))
                #             newString = newString + newHit + ", "
                #     else:
                #         annotatedList.append(annotateData(len(newString), newHit, 'MOVE_PROPERTY'))
                #         newString = newString + newHit + " "

    return

# validateAnnotations([newString, {"entities": annotatedList}], validDict)
# 'entities': [[0, 3, 'STRING_DETAIL'], [8, 14, 'STRING_DETAIL'], [25, 30, 'STRING_DETAIL'], 
def validateAnnotations(data, potentialWords):
    for entsLists in data[1]['entities']:
        if data[0][entsLists[0]:entsLists[1]] not in potentialWords[entsLists[2]]:
            print("FORMATTED INCORRECTLY: [-",data[0][entsLists[0]:entsLists[1]],"-] label = ", entsLists[2], " start ", entsLists[0], " end ", entsLists[1], " ", data)

def generateStringsStringPrePunctuation(listA, listB, listC, label1, label2, label3):
        # move/string where he/she/they do a* punch (into a)* punch (into a)* punch
        random.seed(10)
        allAnnotatedLists = []
        stringMove = ['string', 'move']
        pronoun = ['he', 'she', 'they']
        for word1 in listA:
            for word2 in listB:
                for word3 in listC:
                    pronounNum = random.randint(0,2)
                    if pronounNum == 2:
                        transition = " do a"
                    else:
                        transition = " does a"
                    newString = stringMove[random.randint(0,1)] + " where " + pronoun[pronounNum] + transition

                    big1 = randomWordChance(0,1,1,howListDescriptors, "")
                    big2 = randomWordChance(0,1,1,howListDescriptors, "")
                    big3 = randomWordChance(0,1,1,howListDescriptors, "")

                    jumping1 = randomWordChance(0,1,1,howListPre, "")
                    jumping2 = randomWordChance(0,1,1,howListPre, "")
                    jumping3 = randomWordChance(0,1,1,howListPre, "")

                    intoA1 = randomWordChance(0,1,1, ['into a'], "")
                    intoA2 = randomWordChance(0,1,1, ['into a', 'into another', 'then a'], "")
                    #newString1 = newString + big1 + jumping1 + " " + word1 + "," + intoA1 + big2 + jumping2 + " " + word2 + "," + intoA2 + big3 + jumping3 + " " + word3
                    splitUp = newString.split(" ")
                    annotatedList = []
                    if(splitUp[0] == "string"):
                        annotatedList.append(annotateData(0, 'string', 'STRING_DETAIL'))

                    if(big1 != ""):
                        annotatedList.append(annotateData(len(newString+ " "), big1[1:] , 'LIMB_DESCRIPTION'))
                    newString = newString + big1

                    if(jumping1 != ""):
                        annotatedList.append(annotateData(len(newString + " "), jumping1[1:], 'LIMB_HOW'))
                    newString = newString + jumping1

                    annotatedList.append(annotateData(len(newString + " "), word1 , 'LIMB'))
                    newString = newString + " " + word1 + "," + intoA1
                    
                    if(big2 != ""):
                        annotatedList.append(annotateData(len(newString + " "), big2[1:], 'LIMB_DESCRIPTION'))
                    newString = newString + big2

                    if(jumping2 != ""):
                        annotatedList.append(annotateData(len(newString + " "), jumping2[1:], 'LIMB_HOW'))
                    newString = newString + jumping2

                    annotatedList.append(annotateData(len(newString + " "), word2 , 'LIMB'))
                    newString = newString + " " + word2 + "," +intoA2

                    if(big3 != ""):
                        annotatedList.append(annotateData(len(newString + " "), big3[1:], 'LIMB_DESCRIPTION'))
                    newString = newString + big3

                    if(jumping3 != ""):
                        annotatedList.append(annotateData(len(newString + " "), jumping3[1:], 'LIMB_HOW'))
                    newString = newString + jumping3
                    
                    annotatedList.append(annotateData(len(newString + " "), word3 , 'LIMB'))
                    newString = newString + " " + word3

                
                    allAnnotatedLists.append([newString, {"entities": annotatedList}])    
                    validateAnnotations([newString, {"entities": annotatedList}], validDict)
        addDataToJSONandTxt(allAnnotatedLists)
    
def randomWordChance(start = 0, end = 1, desired = 1, wordIfTrue =[], wordIfFalse = ""):
    randNum = random.randint(start, end)
    if randNum == desired:
        return " " + wordIfTrue[random.randint(0, len(wordIfTrue) - 1)]
    else:
        return wordIfFalse

def annotateData(startIndex, newWord, label):
    return [startIndex, startIndex + len(newWord), label]

def updateJSONData(newData):
    with open('annotationsv2.json', 'r') as file:
        data = json.load(file)
        if isinstance(data['annotations'], NoneType):
            tempList = []
            tempList.append(newData)
            data['annotations'] = tempList
        else:
            tempList = []
            for existingList in data['annotations']:
                tempList.append(existingList)
            tempList.append(newData)
            data['annotations'] = tempList
    os.remove('annotationsv2.json')
    with open('annotationsv2.json', 'w') as file:
        json.dump(data, file)

def updateJSONData2(newData):
    with open('annotationstest.json', 'r') as file:
        data = json.load(file)
        if isinstance(data['annotations'], NoneType):
            tempList = []
            tempList.append(newData)
            data['annotations'] = tempList
        else:
            tempList = []
            for existingList in data['annotations']:
                tempList.append(existingList)
            tempList.append(newData)
            data['annotations'] = tempList
    os.remove('annotationstest.json')
    with open('annotationstest.json', 'w') as file:
        json.dump(data, file)

def addJSONData(newData, filename):
    with open(filename, 'r') as file:
        data = json.load(file)
        if isinstance(data['annotations'], NoneType):
            tempList = []
            for x in newData:
                tempList.append(x)
            data['annotations'] = tempList
        else:
            tempList = []
            for existingList in data['annotations']:
                tempList.append(existingList)
            for x in newData:
                tempList.append(x)
            data['annotations'] = tempList
    os.remove(filename)
    with open(filename, 'w') as file:
        json.dump(data, file)

def Testing():
    with open('annotationsv2.json', 'r') as file:
        data = json.load(file)
        print(type(data['annotations']))

def printSimilarityScores(docs, length):
    i = 0
    j = 0
    avg = 0
    while(i < length):
        j = 0
        while(j <  length):
            if i != j:
                avg = avg + docs[i].similarity(docs[j])
                print("Similarity between ", docs[i], " and ", docs[j], docs[i].similarity(docs[j]))
            j += 1
        print('similarity avg = ', avg / length, 'avg total = ', avg)
        avg = 0
        i += 1

def addDataToJSONandTxt(allAnnotatedLists):
    train_data = open("./raw-data/traindata3.txt","a")
    test_data = open('./raw-data/testdata3.txt', "a")
    train, test =  train_test_split(allAnnotatedLists, test_size =.20, shuffle  = True)
    for item in train:
        train_data.write(item[0] + "\n")
    addJSONData(train, './annotations/v3-annotations.json')
    for item in test:
        test_data.write(item[0] + "\n")
    addJSONData(test, './annotations/v3-test-annotations.json')



In [31]:
#{"classes": ["LIMB", "SPECIAL_LIMB", "LIMB_HOW", "LIMB_DESCRIPTION", "MOVE_TYPE", "MOVE_PROPERTY", "STRING_DETAIL"], "annotations": null}


In [54]:
# cancan dick jab electric flash getup hayashida hellsweep jackhammer junkyard laser scraper matterhorn otgf omen snake edge

# MOVE TYPES power crushes 

# ENTS (LIMB SPECIAL_LIMB LIMB_HOW LIMB_DESCRIPTION MOVE_TYPE MOVE_PROPERTY FRAMES LOWMIDHIGH STRING_DETAIL)

# generate new strings 
# structure: (descriptor) (limb)
# big punch
generateStrings2([howListDescriptors, limbList], 'LIMB_DESCRIPTION', 'LIMB')

# generate new strings 
# structure: (descriptor) (limb)
# punch that launches
generateStrings2a([limbList, tekkenPropertiesPost], 'LIMB', 'MOVE_PROPERTY')

# structure: pre description limb
# jumping punch
generateStrings2([howListPre, limbList], 'LIMB_HOW', 'LIMB')

# generate new strings 
# structure: (tekken p) (move) that (tekken properties post)
# plus on block punch
generateStrings2([tekkenPropertyDescriptors, limbList], 'MOVE_PROPERTY', 'LIMB')

# generate new strings
# structure: limb limb limb string 
# punch punch string
generateStrings2String([limbList, limbList], 'LIMB', 'LIMB')

# generate new strings
# structure: (descriptor) (pre descriptor) (limb)
# big jumping punch
generateStrings3(howListDescriptors, howListPre, limbList, "LIMB_DESCRIPTION", 'LIMB_HOW', 'LIMB')

# generate new strings
# structure: move/moves where he/she/they goes into stance
# goes / does / brings / does that / 

# generate new strings big punch that x
# structure: (howlistdescriptors) (move) that (tekken properties post)
# big punch that launches 
generateStrings3a(howListDescriptors, limbList, tekkenPropertiesPost, 'LIMB_DESCRIPTION', 'LIMB', 'MOVE_PROPERTY')

# generate new strings
# structure: limb () where he/she/they goes into stance
# sword hilt strike into a sword slash 






# generate new strings
# structure: 
# plus on block jumping punch 
# docs = similar(test)
# docs1 = similar(test2)
# printSimilarityScores(docs1, 4)


In [55]:
# generate new strings
# structure: limb limb limb string 
generateStrings3String(limbList, limbList, limbList, 'LIMB', 'LIMB', 'LIMB')

In [56]:
# move/string where he/she/they do a* punch, (into a)* punch, (into a)* punch
generateStringsStringPrePunctuation(limbList, limbList, limbList, 'LIMB', 'LIMB', 'LIMB')

In [57]:
# move/string where he/she/they do a* punch (into a)* punch (into a)* punch
generateStringsStringPre(limbList, limbList, limbList, 'LIMB', 'LIMB', 'LIMB')

In [ ]:
# string detail string detail post 

In [ ]:
# low move* that hits grounded and (high crushes)

In [22]:
# generate new strings
# structure: string/move (that goes/starts with) limb limb limb 

# structure: string where he/she/they (do a)/(does a) (big)/(jumping)/(big jumping) punch punch punch

# structure: move/string where he/she/they do a* punch (into a)* punch (into a)* punch

# structure: move/string 
generateStringsSpecific(string_detail, string_detailPost, tekkenPropertiesPost2, 'str', 'str', 'str')

knockdown, safe two hit moves where the first hit is a mid kick
unblockable two hit move where the first hit is a plus cartwheel
two hit moves where the first hit is a mid chainsaw
punishable two hit moves where the first hit is a kick that knocks down
plus two hit moves where the first hit is a retreating mid kick
two hit strings where the first hit is a approaching unblockable
plus on block two hit moves where the first hit is a high that low crushes
unsafe two hit move where the first hit is a low that evades
two hit strings where the first hit is a unsafe punch that hits grounded
untechable two hit string where the first hit is a punishable backwards low kick that is chargable
two hit moves where the first hit is a jumping high hand
launcher two hit moves where the first hit is a plus punch
minus on block two hit moves where the first hit is a low
two hit strings where the first hit is a rising high
two hit punch elbow move where the first hit is a punch that goes into stance
unblo

In [42]:
# python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy 
#python3 -m spacy train config2.cfg --output ./output-v3 --paths.train ./train2.spacy --paths.dev ./test2.spacy --code specific-moves-factory.py 
#python3 -m spacy train config.cfg --output ./output-v2 --paths.train ./train2.spacy --paths.dev ./test2.spacy --code specific-moves-factory.py 




In [25]:
from types import NoneType


entity_names = ['LIMB', 'SPECIAL_LIMB', 'LIMB_HOW' 'LIMB_DESCRIPTION', 'MOVE_TYPE', 'MOVE_PROPERTY', 'STRING_DETAIL', 'SPECIFIC_MOVE']
train_data = data['annotations']
#train_data = [tuple(i) for i in train_data]
new_list = []

# remove all NoneType data that may have been introduced
i = 0
for x in train_data[:]:
    if isinstance(x, NoneType):
        i+=1
        train_data.remove(x)
print(i)

# format the data into the required format
for i in range(0, len(train_data)):
    new_list = []
    tupled_data = tuple(train_data[i])
    for lists in train_data[i][1].get('entities'):
        new_list.append(tuple(lists))
    new_dict = {
        'entities': new_list
    }
    tupled_data = (train_data[i][0], new_dict)
    train_data[i] = tupled_data

#print(train_data)
test_data = testData['annotations']
new_list = []

# remove all NoneType data that may have been introduced
i = 0
for x in test_data[:]:
    if isinstance(x, NoneType):
        i+=1
        test_data.remove(x)
print(i)
for i in range(0, len(test_data)):
    new_list = []
    tupled_data = tuple(test_data[i])
    for lists in test_data[i][1].get('entities'):
        new_list.append(tuple(lists))
    new_dict = {
        'entities': new_list
    }
    tupled_data = (test_data[i][0], new_dict)
    test_data[i] = tupled_data

0
0


In [26]:
for x in train_data:
    print(x)

for x in test_data:
    print(x)

('untechable five part strings where the first hit is a minus on block mid', {'entities': [(0, 10, 'MOVE_PROPERTY'), (11, 15, 'STRING_DETAIL'), (21, 28, 'STRING_DETAIL'), (39, 44, 'STRING_DETAIL'), (54, 68, 'MOVE_PROPERTY'), (69, 72, 'MOVE_PROPERTY')]})
('tornado, plus four hit move where the first hit is a forward moving dagger', {'entities': [(0, 7, 'MOVE_PROPERTY'), (9, 13, 'MOVE_PROPERTY'), (14, 18, 'STRING_DETAIL'), (38, 43, 'STRING_DETAIL'), (53, 67, 'LIMB_HOW'), (68, 74, 'LIMB')]})
('minus two part moves that ends with a minus tackle that is holdable', {'entities': [(0, 5, 'MOVE_PROPERTY'), (6, 9, 'STRING_DETAIL'), (26, 30, 'STRING_DETAIL'), (38, 43, 'MOVE_PROPERTY'), (44, 50, 'LIMB')]})
('three part moves that begins with a low kick', {'entities': [(0, 5, 'STRING_DETAIL'), (22, 28, 'STRING_DETAIL'), (36, 39, 'MOVE_PROPERTY'), (40, 44, 'LIMB')]})
('staggering plus frame multi hit moves that ends with a safe punch that transitions into a stance', {'entities': [(0, 10, 'MOVE_PROPE

In [27]:
nlp = spacy.load("en_core_web_sm") # load other spacy model
db = DocBin() 

#format data into spacy format
for text, annot in train_data: 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]: 
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity ", span)
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc) 

os.chdir(r'/home/dlin1765/repos/tekken-move-guesser')
db.to_disk("./train2.spacy") 

db1 = DocBin() 
for text, annot in test_data: 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]: 
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity ", span)
        else:
            ents.append(span)
    doc.ents = ents 
    db1.add(doc) 
os.chdir(r'/home/dlin1765/repos/tekken-move-guesser')
db1.to_disk("./test2.spacy") 

In [13]:
import re


@Language.component("specificMoves")
def specificMoves(doc):
    new_ents = []
    expression = "(?:(?:[udbnf]{0,10}|([udbnf]+[\s,.+]*){0,10}|(sway)?|(dpd)?|(ss)?|(ws)?|(fc)?|(wr)?)?[\s,]*[1-4](?:\s*[,+-]\s*[1-4])*)+|((?:[1-4](?:\s*[,+-]\s*[1-4])*)+)"
    for match in re.finditer(expression, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end, label = "SPECIFIC_MOVE")
        if span is not None:
            new_ents.append(span)
    doc.ents = list(doc.ents) + new_ents
    return doc



In [12]:
@Language.component("tekkenSpecific")
def tekkenSpecific(doc):
    nlp = spacy.load("en_core_web_sm")
    matcher_attr = "TEXT" 
    new_ents = []
    matcher = PhraseMatcher(nlp.vocab, attr = matcher_attr)
    props = ['high crushes', 'high crushing', 'high crush','low crush', 'low crushing', 'low crushes', 'guard break', 'grounded', 'guard crush', 'guard crushes', 'guard crushing','flip over', 'in crouch',  'chargable', 'holdable', 'blue spark', 'blue sparks', 'knocks down']
    moves = ['heat burst', 'heat smash', 'chain throw', 'chain grab', 'power crush','homing', 'armor', 'armored', 'parry', 'sabaki', 'instant screw', 'instant tornado', 'rage art', 'heat engager', 'heat engages', 'heat engaging', 'stance', 'transition', 'transitions']
    stances = []
    patterns = [nlp.make_doc(text) for text in props]
    movePatterns = [nlp.make_doc(text) for text in moves]
    matcher.add("PROPERTIES", patterns)
    matcher.add("MOVE_TYPES", movePatterns)
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        # property
        if match_id == 17494066980514271985:
            span = doc.char_span(span.start_char, span.end_char, label = "MOVE_PROPERTY")
        # move type
        elif match_id == 9842506821441691883:
            span = doc.char_span(span.start_char, span.end_char, label = "MOVE_TYPE")
        if span is not None:
            new_ents.append(span)

    doc.ents = list(doc.ents) + new_ents
    return doc

In [11]:
@Language.component("taggerComp")
def taggerComp(doc):
    nlp = spacy.load("en_core_web_sm")
    newDoc = nlp(doc.text)
    i = 0
    for token in doc:
        token.pos_ = newDoc[i].pos_
        token.lemma = newDoc[i].lemma
        token.tag_ = newDoc[i].tag_
        token.dep_ = newDoc[i].dep_
        i += 1
    return doc

In [14]:
import spacy.displacy
#adding a tagger and a parser to  

#nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output/model-best") 


nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output/model-best") 
# new model correctly finds all instances of specific moves
doc8 = nlp1("claudio b2,3,4")
doc9 = nlp1("claudio df2,3,4")
doc10 = nlp1("claudio df 2+3 with outer stuff too")

spacy.displacy.render(doc8, style="ent", jupyter=True)
spacy.displacy.render(doc9, style="ent", jupyter=True)
spacy.displacy.render(doc10, style="ent", jupyter=True)

# kind of messes up certain properties, like "three hit string" if it was 3 hit string it thinks its talking about the 3 MOVE
doc = nlp1("punch string that is safe on block")
doc1 = nlp1("high damage mid punch heat engager")
doc2 = nlp1("three hit string that starts with a low")
doc6 = nlp1("safe on block mid from stance")

spacy.displacy.render(doc, style="ent", jupyter=True) 
spacy.displacy.render(doc1, style="ent", jupyter=True) 
spacy.displacy.render(doc2, style="ent", jupyter=True) 
spacy.displacy.render(doc6, style="ent", jupyter=True)

print("______________________________________________________________________________________________")

nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output-v2/model-best") 
doc = nlp1("punch string that is safe on block")
doc1 = nlp1("high damage mid punch heat engager")
doc2 = nlp1("three hit string that starts with a low")
doc6 = nlp1("safe on block mid from stance")
spacy.displacy.render(doc, style="ent", jupyter=True) 
spacy.displacy.render(doc1, style="ent", jupyter=True) 
spacy.displacy.render(doc2, style="ent", jupyter=True) 
spacy.displacy.render(doc6, style="ent", jupyter=True)
print("______________________________________________________________________________________________")
nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output-v2/model-last") 
doc = nlp1("punch string that is safe on block")
doc1 = nlp1("high damage mid punch heat engager")
doc2 = nlp1("three hit string that starts with a low")
doc6 = nlp1("safe on block mid from stance")
spacy.displacy.render(doc, style="ent", jupyter=True) 
spacy.displacy.render(doc1, style="ent", jupyter=True) 
spacy.displacy.render(doc2, style="ent", jupyter=True) 
spacy.displacy.render(doc6, style="ent", jupyter=True)
print("______________________________________________________________________________________________")
nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output-v3/model-last") 
doc = nlp1("punch string that is safe on block")
doc1 = nlp1("high damage mid punch heat engager")
doc2 = nlp1("three hit string that starts with a low")
doc6 = nlp1("safe on block mid from stance")
spacy.displacy.render(doc, style="ent", jupyter=True) 
spacy.displacy.render(doc1, style="ent", jupyter=True) 
spacy.displacy.render(doc2, style="ent", jupyter=True) 
spacy.displacy.render(doc6, style="ent", jupyter=True)
print("______________________________________________________________________________________________")
nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output-v3/model-best") 
doc = nlp1("punch string that is safe on block")
doc1 = nlp1("high damage mid punch heat engager")
doc2 = nlp1("three hit string that starts with a low")
doc6 = nlp1("safe on block mid from stance")
spacy.displacy.render(doc, style="ent", jupyter=True) 
spacy.displacy.render(doc1, style="ent", jupyter=True) 
spacy.displacy.render(doc2, style="ent", jupyter=True) 
spacy.displacy.render(doc6, style="ent", jupyter=True)

______________________________________________________________________________________________


______________________________________________________________________________________________


______________________________________________________________________________________________


______________________________________________________________________________________________


In [41]:
# results 
# much better results than a few places (just the specific moves part)
# worse at general stuff



In [15]:
nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output/model-last") 
# does identify the stance 
doc3 = nlp1("mid poke into high that transition into a stance")
doc5 = nlp1("stance high homing move")
doc7 = nlp1("jumping ch launch low that goes into stance")
# doesnt identify the stance
doc4 = nlp1("high homing move from stance")

spacy.displacy.render(doc3, style="ent", jupyter=True)
spacy.displacy.render(doc7, style="ent", jupyter=True)
spacy.displacy.render(doc5, style="ent", jupyter=True)
spacy.displacy.render(doc4, style="ent", jupyter=True)
print("______________________________________________________________________________________________")
nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output-v2/model-best") 
doc3 = nlp1("mid poke into high that transition into a stance")
doc5 = nlp1("stance high homing move")
doc7 = nlp1("jumping ch low that goes into stance")
# doesnt identify the stance
doc4 = nlp1("high homing move from stance")
tk = nlp1("move where he does a jumping punch, hits mid")

spacy.displacy.render(doc3, style="ent", jupyter=True)
spacy.displacy.render(doc7, style="ent", jupyter=True)
spacy.displacy.render(doc5, style="ent", jupyter=True)
spacy.displacy.render(doc4, style="ent", jupyter=True)
spacy.displacy.render(tk, style = 'ent', jupyter=True)
print("______________________________________________________________________________________________")
nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output-v3/model-last") 
doc3 = nlp1("mid poke into high that transition into a stance")
doc5 = nlp1("stance high homing move")
doc7 = nlp1("jumping ch low that goes into stance")
# doesnt identify the stance
doc4 = nlp1("high homing move from stance")
tk = nlp1("move where he does a jumping punch, hits mid")

spacy.displacy.render(doc3, style="ent", jupyter=True)
spacy.displacy.render(doc7, style="ent", jupyter=True)
spacy.displacy.render(doc5, style="ent", jupyter=True)
spacy.displacy.render(doc4, style="ent", jupyter=True)
spacy.displacy.render(tk, style = 'ent', jupyter=True)

______________________________________________________________________________________________


______________________________________________________________________________________________


In [18]:
nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output-v2/model-best") 
tekken = nlp1("two hit heat engager that goes mid mid")
t1 = nlp1("jumping plus on block running punch")
t2 = nlp1("four hit stance move where he punches four times")
t3 = nlp1("stance four hit move where he punch four times")
t4 = nlp1("safe on block mid mid string")
spacy.displacy.render(tekken, style="ent", jupyter=True)
spacy.displacy.render(t1, style="ent", jupyter=True)
spacy.displacy.render(t2, style="ent", jupyter=True)
spacy.displacy.render(t3, style="ent", jupyter=True)
spacy.displacy.render(t4, style="ent", jupyter=True)

nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output-v3/model-last") 
print("______________________________________________________________________________________________")
tekken = nlp1("two hit heat engager that goes mid mid")
t1 = nlp1("jumping plus on block running punch")
t2 = nlp1("four hit stance move where he punches four times")
t3 = nlp1("stance four hit move where he punch four times")
t4 = nlp1("safe on block mid mid string")
t5 = nlp1("stance four hit move where he kick four times")
spacy.displacy.render(tekken, style="ent", jupyter=True)
spacy.displacy.render(t1, style="ent", jupyter=True)
spacy.displacy.render(t2, style="ent", jupyter=True)
spacy.displacy.render(t3, style="ent", jupyter=True)
spacy.displacy.render(t4, style="ent", jupyter=True)
spacy.displacy.render(t5, style="ent", jupyter=True)

______________________________________________________________________________________________


In [19]:
t5 = nlp1("stance, safe multihit punch punch strings that ends with a plus on block forward low axekick")
t6 = nlp1("mid attack where he punches down with his fists")
spacy.displacy.render(t5, style="ent", jupyter=True)
spacy.displacy.render(t6, style="ent", jupyter=True)

for token in t6:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)


mid mid ADJ JJ ROOT xxx True False
attack attack NOUN NN ROOT xxxx True False
where where SCONJ WRB ROOT xxxx True True
he he PRON PRP ROOT xx True True
punches punch VERB VBZ ROOT xxxx True False
down down ADP RP ROOT xxxx True True
with with ADP IN ROOT xxxx True True
his his PRON PRP$ ROOT xxx True True
fists fist NOUN NNS ROOT xxxx True False


In [13]:
def taggerComp(doc):
    nlp = spacy.load("en_core_web_sm")
    newDoc = nlp(doc.text)
    i = 0
    for token in doc:
        token.pos_ = newDoc[i].pos_
        token.lemma = newDoc[i].lemma
        token.tag_ = newDoc[i].tag_
        token.dep_ = newDoc[i].dep_
        i += 1
    return doc

for token in t6:
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
                                token.shape_, token.is_alpha, token.is_stop)
print("_____________________________________________________________")
taggerComp(t6)
for token in t6:
        print(token.text,  token.pos_)

mid  ADJ   xxx True False
attack  NOUN   xxxx True False
where  SCONJ   xxxx True True
he  PRON   xx True True
punches  VERB   xxxx True False
down  ADP   xxxx True True
with  ADP   xxxx True True
his  PRON   xxx True True
fists  NOUN   xxxx True False
_____________________________________________________________
mid mid ADJ JJ amod xxx True False
attack attack NOUN NN ROOT xxxx True False
where where SCONJ WRB advmod xxxx True True
he he PRON PRP nsubj xx True True
punches punch VERB VBZ relcl xxxx True False
down down ADP RP prt xxxx True True
with with ADP IN prep xxxx True True
his his PRON PRP$ poss xxx True True
fists fist NOUN NNS pobj xxxx True False
mid ADJ
attack NOUN
where SCONJ
he PRON
punches VERB
down ADP
with ADP
his PRON
fists NOUN


things to add to the model
1. string that (counters) you
2. string that parries you 
3. all the different types of stances 